In [17]:
import pandas as pd

In [18]:
csv_1 = pd.read_csv('Causal.csv', index_col=0, sep='|')

In [19]:
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The correct answer is ""contradiction"". The ..."
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The correct answer is ""neutral"". The premis..."
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The correct answer is ""entailment"". The pre..."
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The correct answer is ""entailment"". The pre..."
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The correct answer is ""entailment"". The pre..."
...,...,...,...
5587,Three women walk down a city street; one looks...,Three women chat at home.,"8. The correct answer is ""neutral"". The cause-..."
5588,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,"9. The correct answer is ""neutral"". The cause-..."
5589,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,"10. The correct answer is ""entailment"". The ca..."
5590,Three naked little boys are playing in a river...,Three boys are playing in mud while one looks on.,Premise: Three naked little boys are playing i...


In [169]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [20]:
prompt_template= '''
Premise: %s, 
Hypothesis: %s.
For the premise '%s' and hypothesis %s, what is the correct answer among the options "entailment", "neutral", "contradiction"? Explain the cause-and-effect relationship between the question and your chosen answer.\n
'''

In [21]:
error = csv_1[csv_1.rationales.isna()]

In [22]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'], x['premise'], x['hypothesis']), axis=1)

In [23]:
error

,premise,hypothesis,rationales,prompt


In [24]:
print(error.prompt[0:].values)

[]


In [25]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [26]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [27]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [14]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['rationales'].lower():
            row.loc['correct_index'][choice] = row['rationales'].lower().index(choice)

In [28]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [29]:
csv_1[csv_1.LLM_answer.isna()]

,premise,hypothesis,rationales,split,correct_index,LLM_answer


In [481]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [30]:
paper = pd.read_csv('../paper.csv', index_col=0)

In [31]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [32]:
paper[~paper.premise.isin(csv_1.premise)]['hypothesis'].values

array([], dtype=object)

In [33]:
paper.shape

(5592, 6)

In [34]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The correct answer is ""contradiction"". The ...","1. the correct answer is ""contradiction"". the ...",{'contradiction': 26},contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The correct answer is ""neutral"". The premis...","2. the correct answer is ""neutral"". the premis...",{'neutral': 26},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The correct answer is ""entailment"". The pre...","3. the correct answer is ""entailment"". the pre...",{'entailment': 26},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The correct answer is ""entailment"". The pre...","4. the correct answer is ""entailment"". the pre...",{'entailment': 26},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The correct answer is ""entailment"". The pre...","5. the correct answer is ""entailment"". the pre...",{'entailment': 26},entailment
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,"8. The correct answer is ""neutral"". The cause-...","8. the correct answer is ""neutral"". the cause-...",{'neutral': 26},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,"9. The correct answer is ""neutral"". The cause-...","9. the correct answer is ""neutral"". the cause-...",{'neutral': 26},neutral
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,"10. The correct answer is ""entailment"". The ca...","10. the correct answer is ""entailment"". the ca...",{'entailment': 27},entailment


In [35]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'

In [36]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The correct answer is ""contradiction"". The ...","1. the correct answer is ""contradiction"". the ...",{'contradiction': 26},contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The correct answer is ""neutral"". The premis...","2. the correct answer is ""neutral"". the premis...",{'neutral': 26},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The correct answer is ""entailment"". The pre...","3. the correct answer is ""entailment"". the pre...",{'entailment': 26},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The correct answer is ""entailment"". The pre...","4. the correct answer is ""entailment"". the pre...",{'entailment': 26},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The correct answer is ""entailment"". The pre...","5. the correct answer is ""entailment"". the pre...",{'entailment': 26},entailment
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,"8. The correct answer is ""neutral"". The cause-...","8. the correct answer is ""neutral"". the cause-...",{'neutral': 26},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,"9. The correct answer is ""neutral"". The cause-...","9. the correct answer is ""neutral"". the cause-...",{'neutral': 26},neutral
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,"10. The correct answer is ""entailment"". The ca...","10. the correct answer is ""entailment"". the ca...",{'entailment': 27},entailment


In [37]:
paper['my_label'] = total_df['LLM_answer']

In [38]:
sum(paper.label == paper.my_label) / len(paper)

0.615343347639485

In [39]:
sum(paper.label == paper.llm_label) / len(paper)

0.699928469241774

In [196]:
total_df.to_csv('causal - full.csv')

In [195]:
csv_1.to_csv('neutral - full.csv', index=False)

In [193]:
csv_1

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,{'contradiction': 194},contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'neutral': 119},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'entailment': 143},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,4. premise: a bald man in an orange shirt and ...,{'entailment': 179},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,5. premise: a boy stands beside a railing at a...,{'entailment': 119},entailment
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. Premise: Three women walk down a city stree...,8. premise: three women walk down a city stree...,{'contradiction': 153},contradiction
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Premise: A basketball player slam dunks the...,9. premise: a basketball player slam dunks the...,"{'entailment': 204, 'entailed': 446}",entailment
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Premise: Two kids wearing white robes are ...,10. premise: two kids wearing white robes are ...,"{'entailment': 249, 'entailed': 567}",entailment
